<a href="https://colab.research.google.com/github/Farbod-gsm99/Farbod-gsm99/blob/main/distilBERT_Cinema_Critic_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install datasets evaluate transformers[sentencepiece]

In [ ]:
from datasets import load_dataset
from transformers import AutoTokenizer, DataCollatorWithPadding
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer

checkpoint = "TheFuriousGunner/distilbert-base-uncased-finetuned-imdb"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2)

In [ ]:
dataset = load_dataset('imdb')

In [ ]:
tokenized_dataset = dataset.map(lambda x: tokenizer(x['text'], truncation=True, padding='max_length', max_length=512), batched=True)

In [ ]:
# tokenized_dataset['train'] = tokenized_dataset['train'].remove_columns(['text'])
# tokenized_dataset['test'] = tokenized_dataset['test'].remove_columns(['text'])
# tokenized_dataset['train'] = tokenized_dataset['train'].rename_column('label', 'labels')
# tokenized_dataset['test'] = tokenized_dataset['test'].rename_column('label', 'labels')

In [ ]:
train_size = 3_000
test_size = int(0.1 * train_size)

downsampled_dataset = tokenized_dataset["train"].train_test_split(
    train_size=train_size, test_size=test_size, seed=42
)
downsampled_dataset

In [ ]:
# train_dataset = tokenized_dataset['train'].shuffle(seed=42)
# eval_dataset = tokenized_dataset['test'].shuffle(seed=42)

In [ ]:
# train_args = TrainingArguments(
#     output_dir='./results',
#     learning_rate=2e-4,
#     per_device_train_batch_size=16,
#     per_device_eval_batch_size=16,
#     num_train_epochs=3,
#     weight_decay=0.01,
#     evaluation_strategy='epoch',
#     save_strategy='epoch',
#     load_best_model_at_end=True,
#     push_to_hub=True,
#     logging_dir='./logs'
# )

training_args = TrainingArguments("test-trainer")

In [ ]:
import evaluate
import numpy as np

metric = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=downsampled_dataset['train'],
    eval_dataset=downsampled_dataset['test'],
    tokenizer=tokenizer,
    data_collator=DataCollatorWithPadding(tokenizer=tokenizer),
    compute_metrics=compute_metrics
)

In [ ]:
trainer.train()

In [ ]:
predictions = trainer.predict(downsampled_dataset['test'])
print(predictions.predictions.shape, predictions.label_ids.shape)

In [ ]:
import numpy as np

preds = np.argmax(predictions.predictions, axis=-1)

In [ ]:
print(preds)
print(predictions.label_ids)

In [ ]:
import evaluate

metric = evaluate.load("accuracy")
metric.compute(predictions=preds, references=predictions.label_ids)